In [143]:
import mxnet as mx
import numpy as np
import sys

In [144]:
class AutoEncoderModel:
    def __init__(self, dims, internal_act=None):
        self.data = mx.symbol.Variable('data')
        self.y = mx.symbol.Variable('label')
        self.fc1_weight = mx.symbol.Variable('fc1_weight')
        self.fc1_bias = mx.symbol.Variable('fc1_bias')
        self.fc2_weight = mx.symbol.Variable('fc2_weight')
        self.fc2_bias = mx.symbol.Variable('fc2_bias')
        x = mx.symbol.FullyConnected(data=self.data, weight=self.fc1_weight,
                                     bias=self.fc1_bias, num_hidden=dims[1])
        self.layer1 = x
        if (internal_act is not None):
            x = mx.symbol.Activation(data=x, act_type=internal_act)
        x = mx.symbol.FullyConnected(data=x, weight=self.fc2_weight,
                                     bias=self.fc2_bias, num_hidden=dims[2])
        self.layer2 = x
        self.loss = mx.symbol.LinearRegressionOutput(data=x, label=self.y)

    def fit(self, data, batch_size, params=None):
        data_iter = mx.io.NDArrayIter(data={'data':data}, label={'label':data},
                batch_size=batch_size, shuffle=True,
                last_batch_handle='roll_over')
        self.model = mx.mod.Module(symbol=self.loss, data_names=['data'], label_names = ['label'])
        self.model.bind(data_shapes=data_iter.provide_data, label_shapes=data_iter.provide_label)
        if (params is not None):
            self.model.set_params(arg_params=params, aux_params=None)
        self.model.fit(data_iter, optimizer_params={'learning_rate':0.005, 'momentum': 0.9},
                num_epoch=50, eval_metric='mse',
                batch_end_callback = mx.callback.Speedometer(batch_size, 2))

In [152]:
def train(data, num_epoc, internal_act=None):
    int_epoc = 100
    params = None
    for i in range(num_epoc/int_epoc):
        model = AutoEncoderModel([data.shape[1], 10, data.shape[1]])
        model.fit(data, int_epoc, params)

        params = model.model.get_params()[0]
        fc1_weight = params.get('fc1_weight').asnumpy()
        fc1_bias = params.get('fc1_bias').asnumpy()
        fc2_weight = params.get('fc2_weight').asnumpy()
        fc2_bias = params.get('fc2_bias').asnumpy()

        np_data = data.asnumpy()
        hidden = np.dot(np_data, fc1_weight.T) + fc1_bias
        output = np.dot(hidden, fc2_weight.T) + fc2_bias
        print("epoc " + str(i * int_epoc) + ": " + str(np.sum(np.square(output - np_data))))
        sys.stdout.flush()
    return params

In [153]:
data1 = mx.ndarray.random_uniform(shape=[10000, 10])
data2 = mx.ndarray.random_uniform(shape=[10, 100])
data = mx.ndarray.dot(data1, data2)
print(data.shape)

(10000L, 100L)


In [158]:
params_linear=train(data, 5000, internal_act=None)

epoc 0: 61822.6


epoc 100: 33974.3


epoc 200: 16870.9


epoc 300: 6809.01


epoc 400: 1039.96


...

epoc 4900: 4.21934e-05


epoc 5000: 4.2044e-05


epoc 5100: 4.19696e-05


epoc 5200: 4.19151e-05


KeyboardInterrupt: 

In [160]:
params_sigmoid=train(data, 5000, internal_act='sigmod')

epoc 0: 63197.2


epoc 100: 38201.0


epoc 200: 18336.1


epoc 300: 7031.36


epoc 400: 2687.0


...

epoc 4500: 4.33602e-05


epoc 4600: 4.21971e-05


epoc 4700: 4.23406e-05


epoc 4800: 4.08596e-05


epoc 4900: 3.99943e-05


In [159]:
params_relu=train(data, 5000, internal_act='relu')

epoc 0: 61618.8


epoc 100: 33887.8


epoc 200: 16133.0


epoc 300: 5691.61


epoc 400: 2538.08


...

epoc 4500: 3.10565e-05


epoc 4600: 3.03081e-05


epoc 4700: 2.95371e-05


epoc 4800: 2.96443e-05


epoc 4900: 2.86874e-05
